In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
#Loading necessary library and dependencies
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding
from llama_index.core import Settings
from dotenv import load_dotenv
import os, json

#Setting API Key
load_dotenv()
api_key=os.getenv("MISTRAL_API_KEY")

#Setting llm and embeddings
llm= MistralAI(api_key=api_key)
embed_model= MistralAIEmbedding(api_key=api_key)

Settings.llm= llm
Settings.embed_model= embed_model


In [3]:
#Structured Extraction
from typing import Literal
from pydantic import BaseModel, Field

class NewsClassificationOutput(BaseModel):
    """News Classification for Narrative Formation."""
    title: str= Field(..., description="The title of the news article.")
    relevance: Literal["Yes", "No"] = Field(..., description="Whether the news is relevant in narrative formation of country or economy.")
    narrative: Literal["Positive", "Negative", "Neutral"] = Field(None, description="Narrative of news, if relevant.")
    confidence: float = Field(..., description="Confidence level (0-1) of relevance and narrative.")
    explanation: str = Field(..., description="Explanation of narrative classification.")

In [4]:
from llama_index.core.llms import ChatMessage

sllm=llm.as_structured_llm(output_cls=NewsClassificationOutput)

In [5]:
#Reading news saved as json on 2025-01-06
date='2025-01-06'
file_path=f'../data/News/{date}/KathmanduPost.json'

with open(file_path, 'r') as json_file:
    news_articles= json.load(json_file)

print(news_articles[0])

{'title': 'Hyatt Centric Soalteemode Kathmandu officially debuts in Nepal', 'link': '/money/2025/01/06/hyatt-centric-soalteemode-kathmandu-officially-debuts-in-nepal', 'description': 'Hyatt Centric Soalteemode Kathmandu invites modern travelers to discover the vibrant essence of Nepal’s capital city through contemporary design, art, and unique experiences.', 'author': 'Post Report'}


In [6]:
# Classify each news article
results = []
for article in news_articles:
    #Prompt
    prompt="""You are tasked with analyzing news articles to determine if they are related to form narratives of Nepalese economy.
    You will be provided with news article from economic section of newspaper and you should help to understand what sorts of narratives is being formed about economy/country.
    For each article:
    1. Indicate whether it is related to form narratives.
    2. If yes, assign an whether the news carry "Positive", "Negative", "Neutral" narratives of country.
    3. Provide a confidence score between 0-1 for correctness of classification.
    4. Explain your reasoning.

    Here is the news article:
    {}
    """

    # Format the query string
    query = f"News Title: {article['title']}\nContent: {article['description']}"
    
    # Populate the prompt with the query string
    input_msg=ChatMessage.from_str(prompt.format(query))

    # Query the structured LLM
    result = sllm.chat([input_msg])
    results.append(json.dumps(result.raw.dict(), indent=2))

In [9]:
for result in results:
    print(result)

{
  "title": "Hyatt Centric Soalteemode Kathmandu officially debuts in Nepal",
  "relevance": "Yes",
  "narrative": "Positive",
  "confidence": 0.9,
  "explanation": "The news article discusses the opening of a new Hyatt hotel in Kathmandu, which is a positive development for the tourism industry in Nepal. This can contribute to economic growth and attract more tourists, thereby forming a positive narrative about the country's economy and tourism sector."
}
{
  "title": "Beijing promises to send more tourists to Nepal. But how?",
  "relevance": "Yes",
  "narrative": "Positive",
  "confidence": 0.85,
  "explanation": "The news article discusses a significant increase in Chinese tourists to Nepal, which is a positive development for Nepal's tourism industry and economy. This influx of tourists can boost economic activity, create jobs, and generate revenue, contributing positively to Nepal's economic narrative."
}
{
  "title": "Interest-free loans draw youths to entrepreneurship",
  "rele

In [10]:
#Saving results
with open(f'../data/News/{date}/narrative-result.json', 'w') as file:
    json.dump(results, file, indent=4)